In [1]:
import os
import sys

from google.colab import drive
drive.mount('/content/drive/')

os.chdir("/content/drive/MyDrive/Colab Notebooks/CS535/Sukavanan_Fusion")
print(f"This is my current working directory --> {os.getcwd()}")
import pandas as pd
import numpy as np
import io
import tarfile
import csv



# global variables
SEQ_LENGTH=20*60*100
BATCH_SIZE=15

Mounted at /content/drive/
This is my current working directory --> /content/drive/MyDrive/Colab Notebooks/CS535/Sukavanan_Fusion


In [2]:
import pickle
# Read metadata (dev+train) labels
labels = pd.read_csv('metadata_mapped.csv')

bin_series= pd.Series(labels['PHQ_Binary'].values, index=labels['Participant_ID'])
reg_series= pd.Series(labels['PHQ_Score'].values, index=labels['Participant_ID'])

In [15]:
# Read test labels
test_labels = pd.read_csv('test_split.csv')

test_bin_series= pd.Series(test_labels['PHQ_Binary'].values, index=test_labels['Participant_ID'])
test_reg_series= pd.Series(test_labels['PHQ_Score'].values, index=test_labels['Participant_ID'])

In [ ]:
# Read train labels
import pickle

train_labels = pd.read_csv('train_split.csv')

train_bin_series= pd.Series(train_labels['PHQ_Binary'].values, index=train_labels['Participant_ID'])
train_reg_series= pd.Series(train_labels['PHQ_Score'].values, index=train_labels['Participant_ID'])

In [2]:
# Read dev labels
dev_labels = pd.read_csv('dev_split.csv')

dev_bin_series= pd.Series(dev_labels['PHQ_Binary'].values, index=dev_labels['Participant_ID'])
dev_reg_series= pd.Series(dev_labels['PHQ_Score'].values, index=dev_labels['Participant_ID'])


In [ ]:
# Total number of depressed in dev+train : 49
# Total number of not-depressed in dev+train: 170

# Total number of depressed in dev: 12
# Total number of not-depressed in dev: 44

# Total number of depressed in train: 37
# Total number of not-depressed in train: 126

# Total number of depressed in test:
# Total number of not-depressed in test:

PHQ_Binary
0    170
1     49
dtype: int64

In [5]:
df_test = df.loc[df['patient_id'].isin(test_labels['Participant_ID'])]

In [3]:
df = pd.read_csv("labeled_vgg16_combined.csv")

In [5]:
df.groupby('patient_id').size()

patient_id
300     648
301     823
302     758
303     985
304     788
       ... 
698    1441
702     730
703     776
707     862
713     790
Length: 219, dtype: int64

In [10]:
df1 = df.loc[df['patient_id'][:100]]

In [13]:
del df

In [40]:
df = pd.read_csv("labeled_vgg16_combined_test.csv")

In [12]:
def padd_dataset(df,bin_series): 
  df_padd=pd.DataFrame(np.zeros((len(bin_series)*SEQ_LENGTH, df.shape[1])), columns=df.columns)

  for i, id in enumerate(list(set(df['patient_id']))):
      curr_df=df.loc[df['patient_id']==id].iloc[:SEQ_LENGTH]
      curr_nrow= curr_df.shape[0] if curr_df.shape[0]<=SEQ_LENGTH else SEQ_LENGTH

      df_padd.iloc[(i+1)*SEQ_LENGTH-curr_nrow: (i+1)*SEQ_LENGTH]= curr_df
  print(df_padd.head(3))
  pi_series = []
  for pi_num in list(set(df['patient_id'])):
      pi_series+= [pi_num]*SEQ_LENGTH

  pi_series = pd.Series(pi_series)
  df_padd['patient_id'] = pi_series
  return df_padd


In [ ]:
df_padd = padd_dataset(df1,test_bin_series)

In [39]:
df_padd.to_csv("updated_test_mfcc_df_padd.csv")

In [ ]:
# DATALOADER
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset_bin(Dataset):

    def __init__(self, df_padd):
        self.df_padd = df_padd

    def __len__(self):
        return self.df_padd.shape[0]//SEQ_LENGTH

    def __getitem__(self, idx):
        drop_cols=['Unnamed: 0', 'name', 'frameTime', 'patient_id', 'bin','req']
        curr_x= self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH].drop(drop_cols, axis=1)
        #print(self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH]['bin'])
        curr_y = np.array(int(self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH]['bin'].iloc[0]))
        #print("this is curr_y ", curr_y)
        X=torch.from_numpy(curr_x.to_numpy()).float()
        y=torch.from_numpy(curr_y)

        return X, y

In [ ]:
class MyDataset_reg(Dataset):

    def __init__(self, df_padd):
        self.df_padd = df_padd

    def __len__(self):
        return self.df_padd.shape[0]//SEQ_LENGTH

    def __getitem__(self, idx):
        drop_cols=['Unnamed: 0', 'name', 'frameTime', 'patient_id', 'bin','req']
        curr_x= self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH].drop(drop_cols, axis=1)
        #print(self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH]['bin'])
        curr_y = np.array(int(self.df_padd.iloc[idx*SEQ_LENGTH: (idx+1)*SEQ_LENGTH]['req'].iloc[0]))
        #print("this is curr_y ", curr_y)
        X=torch.from_numpy(curr_x.to_numpy()).float()
        y=torch.from_numpy(curr_y)

        return X, y

In [ ]:
# Total egemaps datapoints (dev+train)
df = pd.read_pickle('labeled_mfcc_combined.pkl')

In [ ]:
mfcc_df = df

In [ ]:
# Extracting only train egemaps datapoints
train_mfcc_df  = mfcc_df[mfcc_df['patient_id'].isin(train_labels['Participant_ID'])]

In [ ]:
train_mfcc_df.head(50)

In [ ]:
# Extracting only dev egemaps datapoints
dev_mfcc_df  = mfcc_df[egemaps_df['patient_id'].isin(dev_labels['Participant_ID'])]

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = MyDataset_reg(train_egemaps_df) #this is the train data 
valid_data = MyDataset_reg(dev_egemaps_df) #this is the dev data which will be used for validation

# THIS IS DATALOADER FOR BINARY 

# train_data = MyDataset_bin(train_egemaps_df) #this is the train data 
# valid_data = MyDataset_bin(dev_egemaps_df) #this is the dev data which will be used for validation


# valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
# test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))


train_loader = DataLoader(train_data, shuffle=False, batch_size=BATCH_SIZE,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE,drop_last=True)
# test_loader = DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
for X, y in train_loader:
    print("X: ", X.shape, ' y: ', len(y))
    print("---")
    print(y)


X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 4, 2, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([10,  0,  0,  0,  0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([ 0,  0, 22,  0,  0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([7, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 7, 4])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 1, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([ 0,  0,  0, 10,  0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([ 0,  0,  0,  0, 23])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 5, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---
tensor([0, 0, 0, 0, 0])
X:  torch.Size([5, 120000, 23])  y:  5
---

In [ ]:
for X, y in valid_loader:
    print("X: ", X.shape, ' y: ', len(y))
    print("---")

X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---
X:  torch.Size([5, 120000, 23])  y:  5
---


In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print()
print('Sample label size: ', sample_y.size()) # batch_size

Sample input size:  torch.Size([15, 120000, 23])

Sample label size:  torch.Size([15])


In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, output_size, drop_prob):
        super(RNN, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.drop_prob=drop_prob
        self.dropout = nn.Dropout(p=self.drop_prob)

        self.output_size = output_size
        
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.n_layers, 
                            dropout=0.3, batch_first=True)
        

        
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
        self.sig = nn.Sigmoid()
        self.RELU = nn.ReLU()
        

    def forward(self, x, hidden):
        batch_size = x.size(0)

        #x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        #print(lstm_out)
        lstm_out = lstm_out[:, -1, :] # getting the last time step output
        
        out = self.dropout(lstm_out)

        #print("LSTM_out",out)
        out = self.fc(out)
        #print(out)
        out = self.RELU(out)
        #out = self.sig(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:

input_dim = 23
hidden_dim = 64
n_layers = 1
output_size = 1
drop_prob=0.2

net = RNN(input_dim, hidden_dim, n_layers, output_size, drop_prob)

print(net)

RNN(
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(23, 64, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sig): Sigmoid()
  (RELU): ReLU()
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
# h = net.init_hidden(BATCH_SIZE)
# h = tuple([each.data for each in h])
# if(train_on_gpu):
#     sample_x, sample_y = sample_x.cuda(),sample_y.cuda()
# net = net.cuda()
# out,h = net.forward(sample_x,h)

In [ ]:

lr=0.0001

#criterion = costum_loss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
epochs = 20

counter = 0
print_every = 5
clip=5 # gradient clipping
criterion = torch.nn.MSELoss()
if(train_on_gpu):
    net.cuda()
train_loss = 0.
dev_loss = 0.
net.train()
val_h = net.init_hidden(BATCH_SIZE)
for e in range(epochs):

    h = net.init_hidden(BATCH_SIZE)
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        h = tuple([each.data for each in h])

        net.zero_grad()

        output, h = net(inputs, h)
        print(output)
        loss = loss_fn(output.squeeze(), labels.float())
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
        
        if counter % print_every == 0:
            
            dev_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                with torch.no_grad():
                  output, val_h = net(inputs, val_h)
                  dev_loss += torch.sqrt(loss_fn(output.squeeze(), labels.float()))
                  dev_loss += loss.item()

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Train Loss: {:.6f}...".format(train_loss/len(train_loader)),
                  "Train_loder_size {:.6f}".format(len(train_loader)),
                  "Dev Loss: {:.6f}".format(dev_loss/len(valid_loader)))

In [ ]:
import torch
torch.cuda.empty_cache()